In [1]:
import torch
import torch.nn as nn
from model import GlobalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
import numpy as np

In [5]:
global_predictor = torch.load('./results_tokyo/global_predictor_fl.pytorch').cuda(0)

In [6]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [7]:
T = 96
dT = 4
batch_size = 256

In [8]:
mrr_avg_rank_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInTokyoProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    mrr = 0.0
    avg_rank = 0.0
    acc5 = 0.0
    
    with open(filename, 'rb') as f:
        data = pk.load(f)
        data = torch.LongTensor(list(data.values())).cuda(0)
        with torch.no_grad():
            for t in range(T - 2 * dT):
                for i in range(0, data.shape[0], batch_size):
                    data_x = data[i: i + batch_size, t: t + dT]
                    data_t = torch.zeros_like(data_x) + t
                    data_y = data[i: i + batch_size, t + 2 * dT - 1].cpu().numpy()
                    pred = global_predictor.predict(data_x, data_t).cpu().numpy()
                    rank = np.where((np.argsort(-pred, axis=1, ).T - data_y).T == 0)[1] + 1
                    mrr += np.sum(1 / rank)
                    avg_rank += np.sum(rank)
                    acc5 += np.sum(rank <= 5)
            
            sample_size = data.shape[0] * (T - 2 * dT)
            print('Evaluation: mrr = {:.4f}, avg_rank = {:.4f}, acc@5 = {:.4f}'.format(mrr / sample_size, avg_rank / sample_size, acc5 / sample_size))
            mrr_avg_rank_dict[(y, m, d)] = (mrr / sample_size, avg_rank / sample_size, acc5 / sample_size)

/data/fan/UsersInTokyoProcessed/20121201_interp.pk
Evaluation: mrr = 0.7862, avg_rank = 35.1012, acc@5 = 0.8247
/data/fan/UsersInTokyoProcessed/20121202_interp.pk
Evaluation: mrr = 0.8058, avg_rank = 30.5547, acc@5 = 0.8427
/data/fan/UsersInTokyoProcessed/20121203_interp.pk
Evaluation: mrr = 0.7842, avg_rank = 36.1288, acc@5 = 0.8207
/data/fan/UsersInTokyoProcessed/20121204_interp.pk
Evaluation: mrr = 0.7819, avg_rank = 36.1490, acc@5 = 0.8188
/data/fan/UsersInTokyoProcessed/20121205_interp.pk
Evaluation: mrr = 0.7796, avg_rank = 36.4898, acc@5 = 0.8170
/data/fan/UsersInTokyoProcessed/20121206_interp.pk
Evaluation: mrr = 0.7787, avg_rank = 36.9128, acc@5 = 0.8161
/data/fan/UsersInTokyoProcessed/20121207_interp.pk
Evaluation: mrr = 0.7753, avg_rank = 36.8368, acc@5 = 0.8133
/data/fan/UsersInTokyoProcessed/20121208_interp.pk
Evaluation: mrr = 0.7873, avg_rank = 34.8281, acc@5 = 0.8255
/data/fan/UsersInTokyoProcessed/20121209_interp.pk
Evaluation: mrr = 0.8051, avg_rank = 30.6772, acc@5 =

Evaluation: mrr = 0.7935, avg_rank = 34.5681, acc@5 = 0.8284
/data/fan/UsersInTokyoProcessed/20130213_interp.pk
Evaluation: mrr = 0.7925, avg_rank = 34.1830, acc@5 = 0.8281
/data/fan/UsersInTokyoProcessed/20130214_interp.pk
Evaluation: mrr = 0.7905, avg_rank = 34.8838, acc@5 = 0.8259
/data/fan/UsersInTokyoProcessed/20130215_interp.pk
Evaluation: mrr = 0.7893, avg_rank = 34.7373, acc@5 = 0.8250
/data/fan/UsersInTokyoProcessed/20130216_interp.pk
Evaluation: mrr = 0.8039, avg_rank = 32.4230, acc@5 = 0.8392
/data/fan/UsersInTokyoProcessed/20130217_interp.pk
Evaluation: mrr = 0.8224, avg_rank = 27.9171, acc@5 = 0.8579
/data/fan/UsersInTokyoProcessed/20130218_interp.pk
Evaluation: mrr = 0.7958, avg_rank = 34.2150, acc@5 = 0.8301
/data/fan/UsersInTokyoProcessed/20130219_interp.pk
Evaluation: mrr = 0.7935, avg_rank = 34.8299, acc@5 = 0.8279
/data/fan/UsersInTokyoProcessed/20130220_interp.pk
Evaluation: mrr = 0.7901, avg_rank = 34.8637, acc@5 = 0.8257
/data/fan/UsersInTokyoProcessed/20130221_in

In [9]:
with open('./results_tokyo/evaluation_global_fl_rank.pk', 'wb') as f:
    pk.dump(mrr_avg_rank_dict, f)